In [1]:
from llama_index.llms.huggingface import  HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from qdrant_client import QdrantClient, AsyncQdrantClient
from qdrant_client.http import models
from llama_index.vector_stores.qdrant import QdrantVectorStore

from llama_index.core import  VectorStoreIndex, StorageContext, QueryBundle, load_index_from_storage, Settings
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, FilterOperator, FilterCondition

import os
from dotenv import load_dotenv

import nest_asyncio

nest_asyncio.apply()

load_dotenv(r"C:\Users\leduc\OneDrive\Desktop\NLP\grab-capstone-project\NavigaTech\AI_modules\.env")

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


resource module not available on Windows


True

In [29]:
llm = HuggingFaceInferenceAPI(model_name="mistralai/Mistral-7B-Instruct-v0.2", token=os.environ["HF_TOKEN"])
embbeding_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5")

Settings.llm = llm
Settings.embed_model = embbeding_model

C:\Users\leduc\AppData\Local\Temp\ipykernel_13260\2945490377.py:1: DeprecationWarning: Call to deprecated class HuggingFaceInferenceAPI. (Deprecated in favor of `HuggingFaceInferenceAPI` from `llama-index-llms-huggingface-api` which should be used instead.)
  llm = HuggingFaceInferenceAPI(model_name="mistralai/Mistral-7B-Instruct-v0.2", token=os.environ["HF_TOKEN"])
c:\Python311\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\leduc\AppData\Local\llama_index\models--BAAI--bge-large-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need

In [30]:
qdrant_client = QdrantClient(
    url= os.environ["QDRANT_URL"],
    api_key=os.environ["QDRANT_API_TOKEN"],
)

async_qdrant_client = AsyncQdrantClient(
    url= os.environ["QDRANT_URL"],
    api_key=os.environ["QDRANT_API_TOKEN"],
)

In [31]:
qdrant_client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='job_description')])

In [43]:
qdrant_vector_store = QdrantVectorStore(client=qdrant_client, 
                                            aclient = async_qdrant_client,
                                            collection_name="job_description",
                                            dense_vector_name="text-dense",
                                            sparse_vector_name="text-sparse",
                                            enable_hybrid=True,                 
                                        )
qdrant_storage_context = StorageContext.from_defaults(vector_store= qdrant_vector_store)

c:\Python311\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\leduc\.cache\huggingface\hub\models--naver--efficient-splade-VI-BT-large-doc. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_sp

In [44]:
qdrant_index = VectorStoreIndex.from_vector_store(
    qdrant_vector_store,
    use_async = True
)

In [45]:
def filter_constructor(fields):
    if(fields):
        filters = []
        for k, v in fields.items():
            filters.append(MetadataFilter(key = k, value=v))
        
        final_filters = MetadataFilters(
            filters=filters,
            condition=FilterCondition.AND
        )
        return final_filters
    else:
        return []

In [46]:
qdrant_retriever = qdrant_index.as_retriever(similarity_top_k=5, vector_store_query_mode="hybrid")


In [47]:
nodes = qdrant_retriever.retrieve(query)

In [49]:
cohere_rerank = CohereRerank(model="rerank-english-v3.0" ,api_key=os.environ["COHERE_API_TOKEN"], top_n=30)

c:\Python311\Lib\site-packages\pydantic\_internal\_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'validate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [50]:
rerank_nodes = cohere_rerank.postprocess_nodes(nodes, query_bundle=QueryBundle(f"what is best job for given resume {query}"))

In [48]:
for node in nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 2<br>**Similarity:** 0.8348248953077917<br>**Text:** Title: Senior AI Engineer, Speech
Job level: Mid-Level
Working type: fulltime
Company: Zalo
Qualifications & Skills:
At least 3 years of experience building live speech recognition, speech synthesis, or speech enhancement products
Have the ability to mentor members
Passionate about working with speech challenges
Master deep learning frameworks PyTorch, TensorFlow
Experience with TensorRT and TensorFlow Lite is a plus
Strong programming skill in Python, C++ or Java
Strong communication skills and a strong growth mindset

Responsibilities:
Work closely with an active and talent team to research and propose efficient architectures for speech processing challenges
Build high-performance and high-quality speech synthesizer, enhancement and recognition models
Be responsible for data analysis, training, evaluation and related deployment systems
Utilize machine learning technologies for speech processing
Non-stop looking for new state-of-the-art approaches and open-mind to the differences
Stay on time and deliver exceptional products<br>

**Node ID:** 72<br>**Similarity:** 0.7571277656380347<br>**Text:** Title: Machine Learning Data Engineer
Job level: mid-senior level
Working type: fulltime
Company: AS White Global
Qualifications & Skills:
Bachelor s degree in Business, Computer Science, Information Technology, or a related field.
5 10 years of experience in machine learning model development and deployment.
Proficiency in Python and ML frameworks such as TensorFlow, PyTorch, or Keras.
Experience working with cloud platforms AWS preferred, Azure, GCP .
Strong understanding of ML model lifecycle, from training to production deployment.
Hands-on experience with MLOps tools Kubeflow, MLflow, AWS SageMaker .
Familiarity with big data technologies for ML Spark, Hadoop, Kafka .
Strong problem-solving skills and ability to work in cross-functional teams.
Excellent communication skills, with the ability to explain ML concepts to technical and non-technical stakeholders.

Responsibilities:
Build, train, and deploy scalable ML models using AWS SageMaker, TensorFlow, PyTorch, and related frameworks.
Automate ML workflows, ensuring minimal manual intervention and efficient model retraining and monitoring.
Develop and optimize ML pipelines for real-time and batch processing.
Implement MLOps best practices, ensuring seamless model deployment, versioning, and monitoring.
Collaborate with data engineers to integrate ML models into production environments.
Ensure AI governance, compliance, and security standards are met.
Troubleshoot ML model performance issues and optimize for efficiency and scalability.<br>

**Node ID:** 61<br>**Similarity:** 0.5<br>**Text:** Title: Jr/ Mid Data Scientist - Open to Relocation
Job level: not applicable
Working type: fulltime
Company: MTI Technology
Qualifications & Skills:
Be fluent in English.
Have a bachelor s degree in Computer Science, Applied Mathematics, or related fields.
Have at least 2 years of experience in Data Science/Analytics/Computer Vision/NLP, or a similar role.
Proficient in SQL and Python with clear, structured coding.
Familiar with GitHub/GitLab for version control.
Experienced with DS/DA libraries NumPy, Pandas, Scikit-learn, TensorFlow, PyTorch, Seaborn .
Basic NLP skills tokenization, sentiment analysis .
Knowledge of ML basics regression, clustering, forecasting .
Have excellent problem-solving skills.
Be self-motivated and eager to learn new technologies and techniques.
Able to communicate and take direction from team lead.
Experience in Big Data, cloud AWS, Google Cloud, Azure .
Familiar with working with Agile process.
Have a bachelor s degree in Computer Science, Applied Mathematics, or related fields.
Have at least 3-5 years of experience in Data Science/Analytics/Computer Vision/NLP, or a similar role.
Advanced Python skills Pandas, NumPy, Scikit-learn with modular coding.
Proficient in GitHub/GitLab, SQL, and visualization Seaborn, Tableau .
Strong ML expertise random forests, gradient boosting and evaluation metrics.
Skilled in tabular data feature engineering XGBoost, CatBoost .
Proficient in NLP sentiment analysis using NLTK/spaCy/Transformers..
Experienced in fine-tuning LLMs BERT and model serving Flask, FastAPI, Docker .
Familiar with GCP BigQuery, Vertex AI and APIs for ChatGPT/Gemini tasks.
Capable of researching, reading publications, and translating research papers into code.
Have excellent problem-solving skills.
Be self-motivated and eager to learn new technologies and techniques.
Able to collaborate, express and present technical with other team members and stackholder.
Experience in Big Data, cloud AWS, Google Cloud, Azure .
Familiar with working with Agile process.
Experience in leading data science team team size above 2-3 .
Have knowledge or working experience in any domain, such as insurance, logistics, etc.

Responsibilities:
Participate in the entire lifecycle of data science projects
Develop, test, and evaluate data hypotheses, methods, and models using appropriate techniques and tools.
Collaborate with engineers to implement scalable and robust data pipelines and systems.
Stay updated on the latest research and trends in data science.
Participate in the entire lifecycle of data science projects, from problem definition to solution delivery, and maintenance.
Develop, test, and evaluate data hypotheses, methods, and models using appropriate techniques and tools.
Collaborate with engineers to implement scalable and robust data pipelines and systems.
Stay updated on the latest research and trends in data science.
Analyze client data and business needs to identify opportunities and propose data-driven solutions.
Communicate and present your findings and recommendations to clients and stakeholders clearly and effectively.
Lead data science projects as well as provide guidance/mentorship to junior data scientists.<br>

**Node ID:** 67<br>**Similarity:** 0.49091004159433826<br>**Text:** Title: Senior Data Scientist - Open to Relocation
Job level: mid-senior level
Working type: fulltime
Company: MTI Technology
Qualifications & Skills:
Be fluent in English.
Have a bachelor s degree in Computer Science, Applied Mathematics, or related fields.
Have at least 5 years of industrial experience in NLP and/or Machine Learning tabular data .
Expert in Python with production-ready code.
Advanced skills in GitHub/GitLab, SQL, and data visualization.
Proven ML expertise ensemble methods, deep learning using TensorFlow/PyTorch.
Mastery of tabular data XGBoost, PCA and NLP transformers, sequence modeling .
Skilled in training/serving custom LLMs GPT with TensorFlow Serving, FastAPI, Docker/Kubernetes.
Expert in GCP BigQuery, Vertex AI for scalable model deployment/monitoring.
Proficient in using ChatGPT/Gemini for advanced applications e.g., conversational AI with prompt optimization.
Experienced in MLOps and solution design.
Capable of researching, reading publications, and translating research papers into code.
Have excellent problem-solving skills.
Be self-motivated and eager to learn new technologies and techniques.
Be good at communication and presentation: able to communicate with non-tech stakeholders to understand business as well as present technical solutions.
Have experience in leading data science team.
Experience in Big Data, cloud AWS, Google Cloud, Azure .
Familiar with working with Agile process.
A deep understanding of a specific domain or industry vertical insurance, logistics

Responsibilities:
Participate in the entire lifecycle of data science projects, from problem definition to solution delivery, and maintenance.
Develop, test, and evaluate data hypotheses, methods, and models using appropriate techniques and tools.
Collaborate with engineers to implement scalable and robust data pipelines and systems.
Stay updated on the latest research and trends in data science.
Analyze client data and business needs to identify opportunities and propose data-driven solutions.
Communicate and present your findings and recommendations to clients and stakeholders clearly and effectively.
Lead data science projects as well as provide guidance/mentorship to junior, middle data scientists.
Manage multiple projects.<br>

**Node ID:** 58<br>**Similarity:** 0.32133238318232826<br>**Text:** Title: Junior Software Engineer
Job level: internship
Working type: contract
Company: 주식회사 에픽스
Qualifications & Skills:
Solid understanding of Computer Science fundamentals e.g., data structures, algorithms
Strong problem-solving and logical reasoning skills
Basic communication skills in English
Interest in learning Golang, scalable system development, and related tools
Willingness to learn and adapt in a dynamic environment
Effective communication and collaboration skills
Familiarity with Golang experience not required
Exposure to managing high-traffic systems or distributed systems
Experience working with systems that integrate AI models
Background in programming competitions such as ICPC International Collegiate Programming Contest or IOI International Olympiad in Informatics

Responsibilities:
Contribute to designing and building scalable, robust software solutions
Collaborate with the team to solve technical challenges and improve system performance
Learn and apply tools like Bazel, Docker, and other relevant technologies
Support the integration of systems leveraging AI models, as applicable<br>

In [51]:
for node in rerank_nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 72<br>**Similarity:** 0.0087791085<br>**Text:** Title: Machine Learning Data Engineer
Job level: mid-senior level
Working type: fulltime
Company: AS White Global
Qualifications & Skills:
Bachelor s degree in Business, Computer Science, Information Technology, or a related field.
5 10 years of experience in machine learning model development and deployment.
Proficiency in Python and ML frameworks such as TensorFlow, PyTorch, or Keras.
Experience working with cloud platforms AWS preferred, Azure, GCP .
Strong understanding of ML model lifecycle, from training to production deployment.
Hands-on experience with MLOps tools Kubeflow, MLflow, AWS SageMaker .
Familiarity with big data technologies for ML Spark, Hadoop, Kafka .
Strong problem-solving skills and ability to work in cross-functional teams.
Excellent communication skills, with the ability to explain ML concepts to technical and non-technical stakeholders.

Responsibilities:
Build, train, and deploy scalable ML models using AWS SageMaker, TensorFlow, PyTorch, and related frameworks.
Automate ML workflows, ensuring minimal manual intervention and efficient model retraining and monitoring.
Develop and optimize ML pipelines for real-time and batch processing.
Implement MLOps best practices, ensuring seamless model deployment, versioning, and monitoring.
Collaborate with data engineers to integrate ML models into production environments.
Ensure AI governance, compliance, and security standards are met.
Troubleshoot ML model performance issues and optimize for efficiency and scalability.<br>

**Node ID:** 67<br>**Similarity:** 0.0064882697<br>**Text:** Title: Senior Data Scientist - Open to Relocation
Job level: mid-senior level
Working type: fulltime
Company: MTI Technology
Qualifications & Skills:
Be fluent in English.
Have a bachelor s degree in Computer Science, Applied Mathematics, or related fields.
Have at least 5 years of industrial experience in NLP and/or Machine Learning tabular data .
Expert in Python with production-ready code.
Advanced skills in GitHub/GitLab, SQL, and data visualization.
Proven ML expertise ensemble methods, deep learning using TensorFlow/PyTorch.
Mastery of tabular data XGBoost, PCA and NLP transformers, sequence modeling .
Skilled in training/serving custom LLMs GPT with TensorFlow Serving, FastAPI, Docker/Kubernetes.
Expert in GCP BigQuery, Vertex AI for scalable model deployment/monitoring.
Proficient in using ChatGPT/Gemini for advanced applications e.g., conversational AI with prompt optimization.
Experienced in MLOps and solution design.
Capable of researching, reading publications, and translating research papers into code.
Have excellent problem-solving skills.
Be self-motivated and eager to learn new technologies and techniques.
Be good at communication and presentation: able to communicate with non-tech stakeholders to understand business as well as present technical solutions.
Have experience in leading data science team.
Experience in Big Data, cloud AWS, Google Cloud, Azure .
Familiar with working with Agile process.
A deep understanding of a specific domain or industry vertical insurance, logistics

Responsibilities:
Participate in the entire lifecycle of data science projects, from problem definition to solution delivery, and maintenance.
Develop, test, and evaluate data hypotheses, methods, and models using appropriate techniques and tools.
Collaborate with engineers to implement scalable and robust data pipelines and systems.
Stay updated on the latest research and trends in data science.
Analyze client data and business needs to identify opportunities and propose data-driven solutions.
Communicate and present your findings and recommendations to clients and stakeholders clearly and effectively.
Lead data science projects as well as provide guidance/mentorship to junior, middle data scientists.
Manage multiple projects.<br>

**Node ID:** 61<br>**Similarity:** 0.0055119344<br>**Text:** Title: Jr/ Mid Data Scientist - Open to Relocation
Job level: not applicable
Working type: fulltime
Company: MTI Technology
Qualifications & Skills:
Be fluent in English.
Have a bachelor s degree in Computer Science, Applied Mathematics, or related fields.
Have at least 2 years of experience in Data Science/Analytics/Computer Vision/NLP, or a similar role.
Proficient in SQL and Python with clear, structured coding.
Familiar with GitHub/GitLab for version control.
Experienced with DS/DA libraries NumPy, Pandas, Scikit-learn, TensorFlow, PyTorch, Seaborn .
Basic NLP skills tokenization, sentiment analysis .
Knowledge of ML basics regression, clustering, forecasting .
Have excellent problem-solving skills.
Be self-motivated and eager to learn new technologies and techniques.
Able to communicate and take direction from team lead.
Experience in Big Data, cloud AWS, Google Cloud, Azure .
Familiar with working with Agile process.
Have a bachelor s degree in Computer Science, Applied Mathematics, or related fields.
Have at least 3-5 years of experience in Data Science/Analytics/Computer Vision/NLP, or a similar role.
Advanced Python skills Pandas, NumPy, Scikit-learn with modular coding.
Proficient in GitHub/GitLab, SQL, and visualization Seaborn, Tableau .
Strong ML expertise random forests, gradient boosting and evaluation metrics.
Skilled in tabular data feature engineering XGBoost, CatBoost .
Proficient in NLP sentiment analysis using NLTK/spaCy/Transformers..
Experienced in fine-tuning LLMs BERT and model serving Flask, FastAPI, Docker .
Familiar with GCP BigQuery, Vertex AI and APIs for ChatGPT/Gemini tasks.
Capable of researching, reading publications, and translating research papers into code.
Have excellent problem-solving skills.
Be self-motivated and eager to learn new technologies and techniques.
Able to collaborate, express and present technical with other team members and stackholder.
Experience in Big Data, cloud AWS, Google Cloud, Azure .
Familiar with working with Agile process.
Experience in leading data science team team size above 2-3 .
Have knowledge or working experience in any domain, such as insurance, logistics, etc.

Responsibilities:
Participate in the entire lifecycle of data science projects
Develop, test, and evaluate data hypotheses, methods, and models using appropriate techniques and tools.
Collaborate with engineers to implement scalable and robust data pipelines and systems.
Stay updated on the latest research and trends in data science.
Participate in the entire lifecycle of data science projects, from problem definition to solution delivery, and maintenance.
Develop, test, and evaluate data hypotheses, methods, and models using appropriate techniques and tools.
Collaborate with engineers to implement scalable and robust data pipelines and systems.
Stay updated on the latest research and trends in data science.
Analyze client data and business needs to identify opportunities and propose data-driven solutions.
Communicate and present your findings and recommendations to clients and stakeholders clearly and effectively.
Lead data science projects as well as provide guidance/mentorship to junior data scientists.<br>

**Node ID:** 2<br>**Similarity:** 0.005343256<br>**Text:** Title: Senior AI Engineer, Speech
Job level: Mid-Level
Working type: fulltime
Company: Zalo
Qualifications & Skills:
At least 3 years of experience building live speech recognition, speech synthesis, or speech enhancement products
Have the ability to mentor members
Passionate about working with speech challenges
Master deep learning frameworks PyTorch, TensorFlow
Experience with TensorRT and TensorFlow Lite is a plus
Strong programming skill in Python, C++ or Java
Strong communication skills and a strong growth mindset

Responsibilities:
Work closely with an active and talent team to research and propose efficient architectures for speech processing challenges
Build high-performance and high-quality speech synthesizer, enhancement and recognition models
Be responsible for data analysis, training, evaluation and related deployment systems
Utilize machine learning technologies for speech processing
Non-stop looking for new state-of-the-art approaches and open-mind to the differences
Stay on time and deliver exceptional products<br>

**Node ID:** 58<br>**Similarity:** 0.0046455576<br>**Text:** Title: Junior Software Engineer
Job level: internship
Working type: contract
Company: 주식회사 에픽스
Qualifications & Skills:
Solid understanding of Computer Science fundamentals e.g., data structures, algorithms
Strong problem-solving and logical reasoning skills
Basic communication skills in English
Interest in learning Golang, scalable system development, and related tools
Willingness to learn and adapt in a dynamic environment
Effective communication and collaboration skills
Familiarity with Golang experience not required
Exposure to managing high-traffic systems or distributed systems
Experience working with systems that integrate AI models
Background in programming competitions such as ICPC International Collegiate Programming Contest or IOI International Olympiad in Informatics

Responsibilities:
Contribute to designing and building scalable, robust software solutions
Collaborate with the team to solve technical challenges and improve system performance
Learn and apply tools like Bazel, Docker, and other relevant technologies
Support the integration of systems leveraging AI models, as applicable<br>

In [18]:
storage_context = StorageContext.from_defaults(persist_dir=r"C:\Users\leduc\OneDrive\Desktop\NLP\grab-capstone-project\NavigaTech\AI_modules\rag_modules\rag_db")
index =  load_index_from_storage(storage_context = storage_context)

In [19]:
def fusion_retrieve(query, fields, top_k):
    filters = filter_constructor(fields)
    retriever = QueryFusionRetriever(
        [
            qdrant_index.as_retriever(similarity_top_k=top_k, filters=filters ,use_async=True),
            BM25Retriever.from_defaults(
                docstore=index.docstore, similarity_top_k=top_k, language="english"
            ),
        ],
        num_queries=1,
        use_async=True,
        retriever_weights=[0.7, 0.3],
        mode="reciprocal_rerank",
        similarity_top_k=top_k
    )
    return retriever.retrieve(query)

In [12]:
query = """

As an AI Engineer intern specializing in Large Language Models (LLMs), I want to gain hands-on experience fine-
tuning  and  optimizing  NLP,  Generative  AI  models  and  RAG  for  specific  applications.  I  worked  on  building  web
applications,  model  finetuning.  Additionally,  I  also  want  to  explored  cutting-edge  techniques  like  prompt
engineering and transfer learning. I have some experience with HuggingFace, YOLO, Pytorch, Flask, NodeJS and
ReactJS.

EDUCATION

Currently Studying Bachelor of Computer Science
University of Information Technology of Ho Chi Minh city (UIT)

Machine Learning team member of AI club of UIT.

LANGUAGES

IELTS: 6.5 overall

RESEARCH PAPERS & PUBLICATIONS

GPA: 3.5

MMLabUIT at CoMeDi Shared Task: Text Embedding Techniques versus Generation-Based NLI for
Median Judgment Classification.

      Conference: COLING ranking A

Classification of racial stereotypes in Spanish With Ensemble Learning Methods and BERT-based
Adapter Head.
      Conference: IberLEF

 AWARDS & CERTIFICATIONS

Top 3rd CoMeDi competition track 1 of COLING 2025 conference
Top 8th in SemEval 2025 track 9 The Food Hazard Detection Challenge of ACL conference.
Top 9th  DETEST-Dis IberLef 2024 competition track 1 of SEPLN conference
Top 34th in AI City Challenge 2024 competition Track 5
Data Scientist Associate Certificate - Datacamp
IBM Data Science Specialization - IBM
Machine Learning Specialization - DeepLearning.io
SQL (Intermediate) Certificate - Hackerrank

SKILLS & TECH STACK

AI/ML: Pytorch, Tensorflow, scikit-learn, pandas, numpy, matplotlib.
NLP/LLM & RAG/Agent: HuggingFace, LLamaIndex, LangChains, NLTK, Gemini LLM, CohereAI.

Developing multi-query system with different techniques to enhance retrieved content quality.
Can apply Prompt engineering in AI solution with prompt format, prompt techniques.
Conducting new NLP approaches for many problem in real world.
Use different LLM in one system with flexible purpose.
Computer Vision: YOLO(Ultralytics), EasyOCR, Roboflow.

Developing high accuracy and inference speed detection, OCR real-time system.

Back-End: Flask, NodeJS, Postman.

have pretty solid knowledge in CRUD app and intermediate in RESTapi.
Can create end-to-end project integrated with AI/ML solutions.

Front-End: ReactJS, Redux, Streamlit, Gradio, MaterialUI.

have solid skill in developing web UI based on requirements.

Database: MongoDB, PinconeDB, ChromaDB, MySQL.
Research skills: Google Scholar, ACL anathology.

Have more than 1 years experience in researching about NLP/LLM topics and also have research
papers.

PROJECTS

UIT Agent with multi flow answering system

Small Talk Detection: Implements a system to distinguish casual conversation queries and route them
appropriately.
Dynamic Query Transformation: Utilizes HyDE Query Transformation to enhance search performance for
complex queries.
Hybrid Search Integration: Combines BM25 and Semantic Search for precise and contextually relevant
results.
Metadata Filtering and Retrieval: Ensures accurate historical context retrieval through metadata filtering.
Query Re-ranking: Optimizes search result order based on relevance and context using reranking
mechanisms.
HuggingFace LLM Integration: Leverages HuggingFace for advanced natural language understanding and
response generation.
Scalable Architecture: Multi-stage query pipeline with modular end-to-end resolution.
Tech Stack: Pinecone, LlamaIndex, Hugging Face, Gemini, Cohere, Flask, ReactJS.

AI information retrieval Web Application

Text Processing: Extracted token length, conducted sentiment analysis, emotion classification, and
summarized text in English and Vietnamese. Fine-tuned "phoBART-syllable-base" for superior Vietnamese
text summarization with QLora and Quantization techniques.
Image Analysis: Implemented object detection, optical character recognition (OCR), and image captioning
using zero-shot learning models from Ultralytics and EasyOCR.
Audio Recognition: Developed capabilities for audio-to-text conversion and sound classification.

RAG chatbot for course information

Designed and implemented a Retrieval-Augmented Generation (RAG) application utilizing the Vietstral-7B
model from HuggingFace.
Integrated advanced LangChains and CohereAI capabilities to enhance model performance.
Utilized ChromaDB as the retrieval database, employing Cosine similarity and Reranker techniques for
efficient and accurate data retrieval from large datasets.
Enabled context-aware responses by feeding retrieved data into the LLM.
Developed an intuitive user interface for the application using Gradio.

Median Judgment Classification in 7 Languages

Advanced NLP Techniques: Implemented stacked embeddings, averaged embeddings, and Natural
Language Inference (NLI) with BERT-based and generative models.
Custom Token & Data Processing: Enhanced model performance through custom tokens, data
augmentation, and Named Entity Recognition (NER)-based preprocessing.
Evaluation & Optimization: Achieved 0.596 Krippendorff’s α score, significantly improving baseline
classification results.
Machine Learning & NLP: BERT, RoBERTa, XLM-R, BART, Cosine Similarity, NLI
Data Processing: Stratified K-Fold Cross-Validation, Lemmatization, Text Expansion
Development Frameworks: Hugging Face, PyTorch, TensorFlow
Computational Resources: Fine-tuned models on Kaggle P100 GPUs using AdamW & AdaFactor optimizers

Stereotype classification in Spanish

Finetuning BETO, RoBERTa, XLM-RoBERTa, mDeBERTa-v3, DeHATEbert.
Finetuning XLM-RoBERTa-twitter-hate.
Using ensemble learning methods: Hard-voting, Soft-voting, Stacking.
Make use of Adapter Head to improve performance.
 Apply Handcraft-Features for BERT-based models.
 Finetuning SVM, RandomForest with GridSearchCV, cross-validation.

Detecting Violation of Helmet Rule for Motorcyclists

Helmet Rule Violation Detection: Developed an AI-powered system to detect motorcyclist helmet
violations using state-of-the-art object detection models.
Addressed Key Challenges: Tackled unbalanced data and small object detection issues by fine-tuning
YOLOv8 with a p2 head and implementing the SAHI algorithm to enhance precision and recall.
Advanced Object Detection Models: Fine-tuned Real-Time DETR for accurate object detection and
integrated the latest YOLOv9 for improved efficiency.
Deep Learning & Computer Vision: YOLOv8, YOLOv9, DETR, SAHI algorithm
Model Optimization: Fine-tuning, small object detection enhancement, precision-recall improvement
Development Frameworks: PyTorch, TensorFlow, OpenCV



"""

In [23]:
nodes = fusion_retrieve(query=query, fields={"location": "United States"}, top_k= 10)

In [24]:
for node in nodes:
    display_source_node(node, source_length=5000)

**Node ID:** 81f5a833-5bbe-4ec7-8ae7-06f5916481e8<br>**Similarity:** 0.016666666666666666<br>**Text:** ​Senior Applied Scientist | NLP | Large Language Models | Machine Learning | Remote, USWe are looking for multiple Senior Applied Scientists, they are responsible for building AI and machine learning models and pipelines, with a particular emphasis on generative AI, large language models (LLMs), and predictive modeling. This role involves collaborating closely with business and product stakeholders to understand data requirements and product specifications, enabling data-driven decision-making and product design. Additionally, the position requires coordination with software engineers and developers to deliver effective solutions.A key aspect of the role involves rapidly acquiring new tools and technologies, leveraging a strong foundation in machine learning and artificial intelligence alongside basic programming and scripting skills. The individual in this position is expected to have practical experience in the design, development, management, and maintenance of systems as well as in handling large datasets. Expertise with commonly used AI and machine learning frameworks and a solid grasp of fundamental techniques—including deep learning, regression, classification, and clustering algorithms—are essential. Familiarity with retrieval-augmented generation pipelines and designing LLM-supported use cases is also critical.ResponsibilitiesLeading AI strategy by delivering integrated solutions that combine software engineering, statistics, and machine learning for complex clinical applications.Executing rigorous, analytical experiments to achieve incremental improvements using the most suitable techniques.Preparing comprehensive reports and presentations to convey hypotheses and insights that support organizational decision-making.Becoming the subject matter expert on available organizational data by identifying, collecting, transforming, and exploring datasets.Facilitating seamless collaboration between engineers, product analysts, and other teams within the organization.Experience5 to 7 years of relevant experience.Leadership experience is advantageous.At least three years of experience in anapplied scientistrole.Advanced degree (Ph.D. or MSc) in computer science, machine learning, AI, or related fields, or equivalent experience in designing, building, and evaluating machine learning systems.Proficiency with machine learning ecosystem tools such as PyTorch, TensorFlow, Scikit-learn, and XGBoost.Strong statistical analysis and machine learning knowledge, with practical programming skills (e.g., Python, R, SQL).Expertise in data visualization, model implementation, testing, and deployment.Experience evaluating and working with LLM-based pipelines, including retrieval-augmented generation and prompting techniques, is advantageous.Familiarity with tools like LangChain, LlamaIndex, Haystack, Azure AI Studio, vector databases, or equivalent LLM-enabled technologies.Proficiency in database access and management using SQL, Azure Data Factory, or similar technologies.Familiarity with big data frameworks (e.g., Hadoop, Spark) or equivalent systems.Understanding of MLOps principles, including orchestration tools, cloud computing, and observability platforms.💰Up to $220,000 USD + Bonus📍Fully remote anywhere in the USIf you are interested in finding out more about this hire please reach out to jason@enigma-rec.ai​Senior Applied Scientist | NLP | Large Language Models | Machine Learning | Remote, US<br>

**Node ID:** ac64db0a-86a4-4dfb-9f69-3827ae063e99<br>**Similarity:** 0.016666666666666666<br>**Text:** Candidates should have hands-on experience with feature engineering pipelines that enhance the contextual accuracy of LLMs for long-sequence, multi-turn conversational modeling.Advanced experience with embedding manipulation techniques, such as masked token prediction, continuous token augmentation, and variational encoding, to dynamically adjust feature weights based on semantic proximity, particularly in dense retrieval tasks.Mastery of reinforcement learning paradigms for adaptive LLM fine-tuning, utilizing reward functions derived from similarity scores and relevance metrics in high-frequency retrieval systems to improve query-response fidelity.Complex Vector Database Integration & Feature Structuring for Semantic Similarity:Extensive experience designing and engineering feature vectors for semantic search optimization within vector databases (e.g., Pinecone, Faiss, Weaviate) and high-dimensional vector space management. Candidates should have technical proficiency in high-dimensional distance metrics such as cosine similarity, Euclidean distance, and inner product, optimized for specific retrieval tasks.Expert in structuring custom embeddings and vector schemas that enhance semantic search precision by capturing latent contextual signals through PCA, UMAP, and t-SNE techniques. Expertise in embedding normalization, vector scaling, and dimension reduction to balance retrieval speed and vector alignment accuracy.Ability to construct and manipulate KNN (k-nearest neighbor) indices, such as Hierarchical Navigable Small World (HNSW) graphs and IVF (Inverted File) structures, to optimize high-throughput search scenarios. Experience in designing vector clusters using k-means and density-based algorithms, enhancing semantic granularity across vectorized search spaces.High-Precision Semantic & Similarity Search EngineeringMastery in designing and implementing hybrid search architectures that combine dense embeddings with sparse vectors (BM25, TF-IDF) for enhanced retrieval relevance across semantic layers. Experience integrating hybrid dense-sparse models in RAG systems, ensuring that each query achieves optimal precision and recall based on adaptive similarity metrics.Expert in developing and tuning semantic similarity metrics, particularly cosine similarity, for real-time high-volume similarity search tasks. This includes leveraging cosine-similarity-based scoring mechanisms in similarity search pipelines to refine response ranking and ensure contextual relevance.Experience constructing cosine similarity feature transformations to boost search accuracy in query expansion contexts, utilizing cosine-based re-ranking and feature recalibration strategies that enable real-time refinement of relevance scoring.Specialized RAG-Specific Feature Engineering For Retrieval OptimizationExpertise in engineering feature pipelines within RAG architectures, specifically for enhancing query augmentation and retrieval conditioning based on multi-stage RAG frameworks. Candidates should have experience structuring bi-encoder and cross-encoder embeddings to support context-dependent token weighting and real-time relevance adjustments.Advanced experience in developing dynamic re-ranking mechanisms, integrating cosine and dot-product similarity metrics within RAG query layers for optimized retrieval at both coarse-grained and fine-grained levels. Expert in implementing memory-efficient vector stores and cached retrieval pathways that ensure low-latency response in high-frequency applications.Ability to optimize retrieval via custom feature weighting models, which selectively prioritize features based on semantic relevance derived from query intent prediction models, response context preservation, and adaptive relevance feedback mechanisms.Embedding & Similarity Feature Optimization With Real-Time SystemsExpert in designing high-dimensional embedding structures that support rapid cosine similarity calculations, especially within sparse or sparse-dense hybrid retrieval models. Familiarity with optimization techniques that reduce the computational load of similarity calculations in production-grade environments.Expertise in constructing advanced indexing schemes (e.g., IVF, PQ) that facilitate high-speed similarity search across large vector stores while minimizing precision loss. Skilled in implementing and tuning complex scoring layers, including custom cosine similarity scoring models that account for context-switching in multi-turn LLM interactions.Demonstrated capability in applying real-time re-ranking protocols within RAG-based systems, incorporating cosine-similarity-driven reordering, redundancy reduction in query results, and relevance fine-tuning based on LLM contextual embeddings.In addition to AI architecture and AI development, you will be responsible for engineering and implementingend-to-end AI pipelines, ensuring proper integration of these AI models into back-end systems developed usingDjango. You will be heavily i...<br>

**Node ID:** d8decded-d2a7-4f31-bf78-b798273dc179<br>**Similarity:** 0.01639344262295082<br>**Text:** Job Title: Data ScientistLocations: RemoteDuration: 04 Months to startThe HR Tech Insights team is responsible for providing data-driven insights to inform business decisions and drive strategic initiatives. Our Client team leverages advanced analytics, machine learning, and data visualization techniques to uncover hidden patterns and relationships within complex data sets. We are committed to delivering high-quality solutions that enable our stakeholders to make informed decisions and drive business outcomes.Key Responsibilities:Collaborate with cross-functional teams to design and implement Retrieval-Augmented Generation (RAG) pipelines integrating LLMs, vector database and knowledge graphs.Develop and maintain knowledge graphs, including schema design, ontology modeling, and relationship curation to support HR and Legal policy use cases.Partner with stakeholders to identify, structure, and tag content (e.g., policies, FAQs) for ingestion into CaaS platforms.Collaborate on relational-to-graph data migration and implement Graph Neural Network (GNN) embeddings for similarity, clustering, and skill inference.Apply problem-solving skills to analyze complex data sets and identify opportunities for automation, improved insight delivery, or UX optimization.Support conversational AI development by aligning backend graph infrastructure with chatbot/voicebot interface requirements and response accuracy metrics.Stay up-to-date with emerging trends and technologies in AI, graph databases, LLM frameworks, and RAG methods to advise architectural decisions and improve performance.Must Have Skills:3+ years of experience in AI, graph databases, or related fieldStrong problem-solving skills and ability to work with complex data setsExperience with graph database platforms (e.g., Neo4j, Amazon Neptune, Stardog)Experience building or supporting RAG architectures using tools such as LangChain, LlamaIndex, or HaystackFamiliarity with vector databasesExperience implementing with AI frameworks and libraries (e.g., Scikit-learn,TensorFlow, PyTorch, OpenAI)SQL ProficiencyExcellent communication skills and a collaborative mindsetNice to Have:Experience integrating graph and vector-based search into conversational AI pipelines using LLMsExperience with HR data, policy content, or legal documentation in enterprise settingsExperience structuring semi-structured/unstructured documents for graph ingestionFamiliarity with content/document platforms like Confluence, SharePoint, or NotionExperience designing and implementing Graph SolutionsExperience implementing or supporting Retrieval-Augmented Generation (RAG) architectures using frameworks like LangChain or LlamaIndexExperience integrating knowledge graphs and vector stores with conversational AI platforms using LLMs (e.g., OpenAI, LLaMa, Claude) for enterprise workflows.Experience with Content as a Service knowledge graphExperience in knowledge graph schema design, ontology modeling, and maintaining data governance for graph-based systems.Familiarity with structuring and tagging policy documents from platforms like Confluence or SharePoint for ingestion into knowledge graphs.Knowledge of HR technology and data analyticsFamiliarity with cloud-based platforms (e.g., AWS, GCP, Azure)Python Proficiency<br>

**Node ID:** f19b6f18-d065-4a76-b387-64c533ba742f<br>**Similarity:** 0.01639344262295082<br>**Text:** Salary: $180k to $240k + bonus + equityHybrid positionWe are seeking a highly skilled Data Scientist with a focus on Large Language Models (LLMs) and Machine Learning (ML) to join our team. The ideal candidate will have deep experience in developing and deploying AI solutions, particularly using LLMs such as GPT-3, BERT, or similar technologies, to solve complex business problems. You will work closely with cross-functional teams to design, build, and deploy data-driven models that leverage advanced language processing techniques and traditional ML methods to drive innovation and business success.Key Responsibilities:Large Language Model (LLM) Development:Design, develop, and deploy LLM-based applications using technologies like GPT-3, BERT, or similar.Implement AI solutions using LLMs to automate and optimize business processes, such as text generation, summarization, sentiment analysis, and document classification.Fine-tune LLMs to improve model accuracy and efficiency for business-specific tasks.Machine Learning Model Development:Develop and deploy machine learning models using various algorithms (e.g., regression, classification, clustering) and deep learning techniques.Integrate LLM-based solutions with traditional machine learning models for complex, multi-faceted business problems.Algorithm Development and Optimization:Build and optimize algorithms to handle large-scale data and improve model performance.Leverage ML frameworks such as TensorFlow, PyTorch, and Scikit-learn to implement and fine-tune models, with a particular focus on language processing tasks.Python Programming and Software Development:Write clean, efficient, and scalable Python code for LLM and machine learning model development.Utilize libraries such as Pandas, NumPy, and SciPy to manipulate and analyze data, ensuring the smooth integration of models into production environments.Model Evaluation and Validation:Evaluate model performance using appropriate metrics (e.g., accuracy, precision, recall, F1 score) and ensure robustness, reliability, and generalizability of LLM and ML models.Continuously improve model outcomes by iterating on performance and fine-tuning for specific use cases.Collaboration and Reporting:Work closely with business leaders, engineers, and other stakeholders to translate business requirements into LLM and ML solutions.Present findings and model results to non-technical stakeholders, ensuring actionable insights.Create and maintain detailed documentation for processes, models, and methodologies.Requirements:Proven experience as a Data Scientist or Machine Learning Engineer with a focus on Large Language Models (LLMs).Strong proficiency in Python and experience with libraries such as Pandas, NumPy, TensorFlow, PyTorch, and Hugging Face Transformers.Experience in deploying and fine-tuning LLMs (e.g., GPT-3, BERT) for various NLP tasks such as text generation, summarization, sentiment analysis, and document processing.Solid understanding of machine learning algorithms and AI techniques, including supervised and unsupervised learning, deep learning, and reinforcement learning.Experience with data visualization tools (e.g., Matplotlib, Seaborn, Plotly) for presenting insights and findings.Experience with cloud platforms and big data technologies (e.g., AWS, Google Cloud, Hadoop, Spark) is a plus.Familiarity with version control systems like Git.Bachelor’s or Master’s degree in Computer Science, Data Science, Mathematics, or a related field.Preferred Skills:Experience with LLM Integration:Hands-on experience in building end-to-end pipelines for LLM-based solutions, integrating models into larger workflows, and deploying them in production environments.Familiarity with frameworks such as Hugging Face, OpenAI API, or similar tools for LLM development and deployment.Experience in advanced NLP tasks, including Named Entity Recognition (NER), sentiment analysis, text classification, and question answering.Strong communication skills with the ability to explain complex LLM and ML concepts to non-technical audiences.Familiarity with agile methodologies and working in a collaborative team environment.<br>

**Node ID:** 5820e44e-a039-4a27-8d35-68d858002b34<br>**Similarity:** 0.016129032258064516<br>**Text:** Join a team recognized for leadership, innovation and diversityThe Future Is What You Make It.When you join Honeywell, you become a member of our global team of thinkers, innovators, dreamers, and doers who make the things that make the future. That means changing the way we fly, fueling jets in an eco-friendly way, keeping buildings smart and safe and even making it possible to breathe on Mars.Working at Honeywell isn’t just about developing cool things. That’s why our employees enjoy access to dynamic career opportunities across different fields and industries.Are you ready to help us make the future?About The RoleJoin a team that designs, develops, and integrates highly complex software applications within Honeywell. As an Artificial Intelligence, Machine Learning Intern here at Honeywell, you will have the opportunity to gain hands-on experience and contribute to cutting-edge AI projects. You will work closely with our team of talented AI engineers and researchers to develop and implement innovative solutions. This internship will provide you with valuable insights into the field of AI and machine learning, and you will have the chance to apply your knowledge in a real-world setting.What You Will LearnModel, build and test AI software to ensure it can take large swatch of data and achieve desired resultsUse Python, JavaScript and C++ to create a faster, more capable AIEvaluate statistics, data and algorithms to increase productivity of assets, people and processes in industrial applications/use casesWork with team of machine-learning and data engineers to develop seamless AI and integrationStay current on AI knowledge, trends, and regulationsCollect, organize, and present progress with team leadership and stakeholdersThrough hands-on learning experiences, global exposure, networking, and professional development opportunities, Honeywell interns will shape the future. You’ll have the opportunity to work alongside industry experts, lead initiatives that refine technical skills, and have unparalleled mentorship and growth opportunities that will elevate your career.#FutureshaperLocation:This position is available in multiple locations across the United States.Pay Range:The hourly base salary range for this position is $25.00/hr - 38.50/hr. Please note that this salary information serves as a general guideline. Honeywell considers various factors when extending an offer, including but not limited to the scope and responsibilities of the position, the candidate's work experience, education and training, key skills, as well as market and business considerations.Minimum QualificationsMust be currently pursuing a bachelor’s degree (or higher) from an accredited college or university in Engineering, Computer Science or similar fieldMust graduate in December 2025 or laterMust continue enrollment in degree program upon completion of the internshipValidated academic excellence (3.0 GPA and higher)We ValueUnderstanding of data science, linear algebra, algorithms, and statisticsDemonstrated ability to develop efficient and high-quality software and in-depth knowledge of computer programming language such as Python, JavaScript, and C++Experience in developing and deploying end-to-end AI/ML solutionsUnderstanding of data warehousingWillingness to learn and continue developing knowledge in an up-and-coming fieldExcellent problem-solving skills with the ability to thrive in a demanding fast-paced work environmentStrong interpersonal and communication skills and a willingness to collaborate cross-functionally with different teamsBenefits Of Interning For HoneywellInterning at Honeywell provides hands-on experience with cutting-edge technologies, professional development, and mentorship within a global organization. Interns will receive relocation benefits and short-term housing if they are 50 miles or more from their assigned work location (excluding Puerto Rico and Canada).The application period for the intern position is estimated to be through the end of March 2025; however, this may be shortened or extended depending on business needs and the availability of qualified candidates.HoneywellURNAMAdditional InformationJOB ID: req460209Category: EngineeringLocation: United StatesNonexemptEarly Career (ALL)Honeywell is an equal opportunity employer. Qualified applicants will be considered without regard to age, race, creed, color, national origin, ancestry, marital status, affectional or sexual orientation, gender identity or expression, disability, nationality, sex, religion, or veteran status.<br>

**Node ID:** d703c39b-a3fd-43de-a4df-80bad5bd9d43<br>**Similarity:** 0.016129032258064516<br>**Text:** Your work will focus on enhancing user experiences by embedding natural language understanding into practical applications and improving real-time data processing.Key Responsibilities:Develop and optimize workflows for RAG and NLP models.Build and maintain scalable vector databases for AI applications.Design and implement applications that integrate with large language models via APIs, such as ChatGPT, for user interaction, query processing, and response generation.Collaborate with cross-functional teams to integrate AI capabilities into various systems.Conduct experiments to fine-tune models and enhance their performance.Stay updated on the latest trends and advancements in AI technologies.Required Qualifications:Bachelor’s degree in Computer Science, AI, or a related field.1-3 years of experience working with RAG, NLP, and vector databases.Proficiency in Python and experience with frameworks such as PyTorch, TensorFlow, or Hugging Face.Experience developing applications that interact with large language models.Familiarity with vector databases (e.g., Pinecone, Weaviate).Experience with real-time data retrieval and processing through APIs like ChatGPT.Strong understanding of natural language processing techniques and text embedding models.Desired Qualifications:Advanced degree in a relevant field.Experience deploying AI models in production environments.Familiarity with reinforcement learning for large language models and cloud platforms.Experience with frameworks like LangChain or Pydantic AI, or custom LLM frameworks.Experience with RAG applications, including locally hosted or public LLM API services.Experience with LLM tool calling and fine-tuning models for tasks like sentiment analysis and text classification.Expertise in language model prompt engineering.If you’re excited about building AI-driven applications and working with cutting-edge technologies, we’d love to hear from you!<br>

**Node ID:** e85f2d7b-ea74-43f9-8da8-13580d33b917<br>**Similarity:** 0.015873015873015872<br>**Text:** Director of Applied AIDirector, Applied ScienceDirector, ML ScienceLocation: Remote USAOur client, a global leader in technology solutions for financial and professional services industries, is seeking an experienced AI leader to drive innovation and deliver transformative AI capabilities across their enterprise product suite. This role will oversee the deployment of cutting-edge machine learning (ML) models and frameworks, ensuring seamless integration with existing systems while advancing the organization’s AI strategy.Key ResponsibilitiesAI Model Deployment:Lead the deployment of advanced AI models into production environments, integrating them with core systems such as Search, Discovery, Recommendation Engines, and Natural Language Processing (NLP) applications.Technical Leadership:Lead the technical direction and mentor a team of ML researchers and engineers to design, develop, and deploy state-of-the-art AI/ML solutions that enhance product functionality and user experience.Enterprise AI Frameworks:Ensure AI engineering frameworks, ML systems, data pipelines, and security protocols are optimized for scalability and align with enterprise priorities.Stakeholder Engagement:Collaborate with technical and business leaders, including executives and subject matter experts (SMEs), to advocate for AI initiatives, model integrations, and deployment strategies that drive business value.Advanced Retrieval Techniques:Implement cutting-edge retrieval methods to maximize the potential of Large Language Models (LLMs) within enterprise environments for enhanced data discovery and decision-making.Semantic Information Retrieval Systems:Establish robust semantic information retrieval frameworks using tools like Azure Cognitive Search, LucidWorks/Solr 9, Google Cloud Search, or AWS Kendra to enable enterprise-wide access to structured knowledge.Policy Implementation:Uphold company policies and IT best practices in all aspects of AI development and deployment.QualificationsTechnical Expertise:Extensive hands-on experience developing advanced AI capabilities, including large language models (LLMs) and other state-of-the-art technologies.Leadership Experience:Proven track record managing technical organizations as a Director, VP, or CTO, with experience leading diverse teams across global portfolios.Domain Knowledge:Broad exposure to various industries leveraging AI technologies, including e-commerce, finance, SaaS, and more.Architectural Understanding:Deep knowledge of AI architectures such as Retrieval-Augmented Generation (RAG) and database systems critical for scalable solutions.Global Impact:Demonstrated success in solving complex technical challenges related to AI delivery while driving thought leadership and measurable business impact at scale.Implementation Expertise:Experience delivering large-scale AI solutions across complex projects with a focus on enterprise adoption and operational excellence.This is a unique opportunity to lead transformative AI initiatives within a multinational organization committed to innovation. Join us to shape the future of applied AI while delivering impactful solutions that redefine industry standards.<br>

**Node ID:** e4506d74-8a55-4b01-aba0-6dcc5a608aff<br>**Similarity:** 0.015873015873015872<br>**Text:** This role will involve working on various projects, including text analysis, language modeling, and model deployment in production environments. You will also be instrumental in applying generative models for creative and business purposes, such as text generation and data augmentation.Primary Responsibilities/Essential FunctionsNatural Language Processing (NLP):Lead the design and implementation of advanced NLP models for tasks such as text classification, named entity recognition (NER), topic modeling, sentiment analysis, and language translation.Apply cutting-edge deep learning techniques like Recurrent Neural Networks (RNNs), Long Short-Term Memory (LSTMs), and transformer models (e.g., BERT, GPT) for complex NLP tasks.Leverage pre-trained language models, word embeddings (Word2Vec, GloVe, FastText), and fine-tune them to meet custom business requirements.Generative AI:Apply Generative AI techniques, such as Generative Adversarial Networks (GANs), Variational Autoencoders (VAEs), and transformer-based models (e.g., GPT-3, T5), to develop solutions for text generation, data augmentation, and other creative use cases.Explore innovative applications of Generative AI in content creation, including summarization, question generation, and dialogue systems.Stay up-to-date with the latest advancements in Generative AI, and integrate them into existing pipelines to enhance model performance and functionality.Collaborate with cross-functional teams to explore new business applications for generative models, such as synthetic data generation for model training or content generation for marketing.Supervised Learning:Develop and optimize machine learning models using supervised learning techniques such as regression, classification, Support Vector Machines (SVMs), and decision trees.Evaluate models using performance metrics such as accuracy, precision, recall, F1 score, and use cross-validation to ensure model robustness.Unsupervised Learning:Use clustering algorithms, such as K-means and hierarchical clustering, and dimensionality reduction techniques like Principal Component Analysis (PCA) to uncover hidden patterns in data.Implement anomaly detection models to identify rare events or outliers in datasets, supporting business intelligence and decision-making.Deep Learning:Lead the design and deployment of deep learning models using Convolutional Neural Networks (CNNs), RNNs, LSTMs, and transformers to handle complex tasks in both NLP and Generative AI.Optimize and fine-tune deep learning architectures to improve accuracy, performance, and scalability of models in production.Model Deployment and MLOps:Deploy machine learning models into production using cloud platforms such as Azure ML, ensuring scalability and performance.Implement MLOps best practices, including CI/CD pipelines, model versioning, and automated retraining using tools like MLflow, Kubeflow, and Azure ML Pipelines.Monitor models post-deployment, track model performance, identify drift, and retrain models as necessary to maintain their relevance and accuracy.QualificationsEducation:Master’s or PhD in Computer Science, Data Science, Mathematics, Statistics, Engineering, or a related field.Experience:5+ years of experience in machine learning, with a focus on Natural Language Processing (NLP) and Generative AI.Extensive experience with deep learning frameworks such as TensorFlow, Keras, or PyTorch for building and deploying models.Proven expertise in deploying models to production environments using cloud platforms such as Azure ML, AWS, or GCP.Familiarity with MLOps practices, including CI/CD, model versioning, and automated retraining.Technical Skills:Proficiency in Python and machine learning libraries such as scikit-learn, TensorFlow, PyTorch, Keras, Hugging Face Transformers, NLTK, and SpaCy.Expertise in NLP techniques, including tokenization, word embeddings, transformers (e.g., BERT, GPT), and sequence models like RNNs and LSTMs.Experience with Generative AI models such as GANs, VAEs, and transformers (e.g., GPT-3, T5) for tasks like text generation and data augmentation.Experience with cloud platforms for model deployment, including Azure ML, AWS, or GCP.Familiarity with MLOps tools such as MLflow, Kubeflow, and Azure ML Pipelines for tracking, monitoring, and automating ML models.Leadership and Soft Skills:Excellent communication skills with the ability to explain complex technical concepts to non-technical stakeholders.Collaborate with data engineers, product managers, and other stakeholders to translate business requirements into machine learning solutions.<br>

**Node ID:** 9d96a51e-0599-4c4e-b9c8-657e22adbc20<br>**Similarity:** 0.015625<br>**Text:** Role Overview:Become the guardian of quality and reliability for groundbreaking AI. As an AI Testing Engineer at LLABS, you'll be responsible for rigorously evaluating and validating our Cognitive Agentic Framework (CAF) and the autonomous agents it creates. You won't just run test scripts; you'll devise creative strategies to probe the limits of complex AI systems, identify subtle failure modes, benchmark performance, and ensure our agents behave reliably and ethically in real-world scenarios.Responsibilities:Design, develop, and execute innovative test plans and strategies specifically tailored for evaluating complex, potentially non-deterministicagentic AI systems.Implement and maintain automated testing frameworks (using Python) to ensure consistent and efficient validation of the CAF and agent behaviors.Conduct rigorous performance benchmarking, stress testing, and scalability assessments across different deployment environments (cloud, decentralized, potentially on-premise).Identify, analyze, meticulously document, and track bugs, anomalies, and edge cases within the AI framework and specific agents.Collaborate closely with AI and Backend Engineers to understand system architecture, identify potential weaknesses, and integrate testing early in the development cycle.Develop and utilize metrics and evaluation criteria appropriate for assessing AI agent performance, reliability, and safety.Help define and enforce quality standards, ensuring transparency and traceability through rigorous documentation and reporting.Qualifications (Must-Haves):Proven experience in software testing, with a strong interest or direct experience in testingAI/ML systems.Proficiency inPythonfor scripting and test automation.Experience with test automation frameworks and tools.Strong analytical skills and exceptional attention to detail – ability to think critically about how complex systems might fail.Understanding of fundamental AI/ML concepts and challenges related to validation and evaluation.Excellent communication skills for documenting issues clearly and collaborating with engineering teams.A proactive and tenacious approach to finding and diagnosing problems.Qualifications (Nice-to-Haves):Experience testing complex distributed systems.Familiarity with AI benchmarks and evaluation methodologies (e.g., HELM, specific agent benchmarks).Understanding of concepts related to AI safety, ethics, and bias testing.Experience with performance testing tools and techniques.Familiarity with CI/CD pipelines and integrating automated tests.Why Join Us?Tackle the unique and exciting challenge of ensuring the quality of cutting-edge autonomous AI agents.Develop novel testing methodologies for a rapidly evolving field.Play a critical role in building trust and reliability for our enterprise clients.Work directly with the core engineering team in a fast-paced, impactful environment.Ground-floor opportunity in a well-funded startup with significant enterprise traction.<br>

**Node ID:** 7f8989b8-48fe-4698-bcd9-0516351671fd<br>**Similarity:** 0.015625<br>**Text:** Our Conversational AI Platform instantly improves your customers’ communications experience. IntelePeer provides industry leading time-to-value with Agentic AI solutions that work seamlessly with your infrastructure. Our no-code templates, low-code, co-creation, and developer API options provide you with simple, easy-to-use tools that can be utilized by anyone.What we are looking for:As a Senior Machine Learning Engineer with a focus on NLP, you will lead the design, development, and optimization of state-of-the-art transformer-based models. You will play a key role in the full ML lifecycle, including data preprocessing, model building, fine-tuning, transfer learning, evaluation, and deployment. You will also be responsible for ensuring model performance and fairness by addressing biases and ensuring ethical AI practices.Key Responsibilities:Design, implement, and optimize NLP models with a strong focus on transformer-based architectures (e.g., BERT, GPT, T5).Build and fine-tune models for a variety of NLP tasks, such as text classification, sentiment analysis, named entity recognition (NER), machine translation, and question answering.Develop and implement strategies for transfer learning to leverage pre-trained models effectively in various domains.Lead the evaluation of model performance using metrics such as accuracy, precision, recall, and F1 scores, while ensuring robustness and fairness in predictions.Conduct research to detect, mitigate, and manage biases in ML models, ensuring ethical and transparent AI solutions.Optimize model inference and ensure low-latency performance in real-time applications.Mentor junior engineers and provide technical leadership in ML/NLP best practices.Stay current with the latest advancements in NLP, transformer architectures, and ML techniques to drive continuous improvement.Requirements:Proven expertise in NLP with at least 5 years of hands-on experience in building and deploying NLP models.Strong knowledge of transformer-based models (e.g., BERT, GPT, RoBERTa, T5, etc.)<br>